<a href="https://colab.research.google.com/github/abarb2022/Facial-Expression-Recognition-Challenge/blob/main/loading_kaggle_data_to_colab%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/yEXkEUqK52Q

**Downloading Kaggle data sets directly into Colab**

Install the kaggle python library

In [2]:
! pip install kaggle

Mount the Google drive so you can store your kaggle API credentials for future use

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make a directory for kaggle at the temporary instance location on Colab drive.

Download your kaggle API key (.json file). You can do this by going to your kaggle account page and clicking 'Create new API token' under the API section.

In [4]:
! mkdir ~/.kaggle

If you want to copy the kaggle API credentials to the temporary location... (I recommend placing it on your Google Drive)

In [ ]:
#! cp kaggle.json ~/.kaggle/

Upload the json file to Google Drive and then copy to the temporary location.

In [5]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

Change the file permissions to read/write to the owner only

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

**Competitions and Datasets are the two types of Kaggle data**

**1. Download competition data**

If you get 403 Forbidden error, you need to click 'Late Submission' on the Kaggle page for that competition.

In [7]:
! kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 90% 256M/285M [00:00<00:00, 812MB/s] 
100% 285M/285M [00:00<00:00, 842MB/s]


Unzip, in case the downloaded file is zipped. Refresh the files on the left hand side to update the view.

In [8]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.metrics import accuracy_score, classification_report


In [10]:
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')
data_df = pd.read_csv('icml_face_data.csv')

# Let's examine the structure
print(data_df.head())
print("\nUsage distribution:")
print(data_df[' Usage'].value_counts())

   emotion     Usage                                             pixels
0        0  Training  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  Training  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  Training  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  Training  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  Training  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...

Usage distribution:
 Usage
Training       28709
PublicTest      3589
PrivateTest     3589
Name: count, dtype: int64


In [11]:
# Define dataset class with data augmentation
class FacialExpressionDataset(Dataset):
    def __init__(self, dataframe, usage_type='Training', transform=None):
        self.data = dataframe[dataframe[' Usage'] == usage_type]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx][' pixels']
        pixels = np.array([int(pixel) for pixel in pixels.split()], dtype=np.uint8)
        image = pixels.reshape(48, 48, 1)  # 48x48x1

        # Convert to tensor and normalize
        image = transforms.ToTensor()(image).float()
        image = transforms.Normalize(mean=[0.5], std=[0.5])(image)

        if self.transform:
            image = self.transform(image)

        label = self.data.iloc[idx]['emotion']
        return image, label

# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
])

In [12]:

train_dataset = FacialExpressionDataset(data_df, 'Training', train_transform)
val_dataset = FacialExpressionDataset(data_df, 'PublicTest')
test_dataset = FacialExpressionDataset(data_df, 'PrivateTest')

print(f"\nDataset sizes:")
print(f"Training: {len(train_dataset)} samples")
print(f"Validation: {len(val_dataset)} samples")
print(f"Test: {len(test_dataset)} samples")


Dataset sizes:
Training: 28709 samples
Validation: 3589 samples
Test: 3589 samples


In [13]:
# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
# Define a more sophisticated model
class ImprovedEmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(ImprovedEmotionCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = ImprovedEmotionCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)

Using device: cuda


In [16]:
# Training loop with early stopping
num_epochs = 30
best_val_accuracy = 0.0
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total

    scheduler.step(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.4f}')

    # Early stopping and model checkpointing
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved new best model")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping after {patience} epochs without improvement")
            break

# Load best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on test set
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Generate classification report
print("\nTest Set Performance:")
print(classification_report(all_labels, all_preds, target_names=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']))
print(f"Test Accuracy: {accuracy_score(all_labels, all_preds):.4f}")

Epoch 1/30:
Train Loss: 1.5698 | Train Acc: 0.3944
Val Loss: 1.3307 | Val Acc: 0.4840
Saved new best model
Epoch 2/30:
Train Loss: 1.2780 | Train Acc: 0.5140
Val Loss: 1.1752 | Val Acc: 0.5528
Saved new best model
Epoch 3/30:
Train Loss: 1.1863 | Train Acc: 0.5479
Val Loss: 1.1531 | Val Acc: 0.5653
Saved new best model
Epoch 4/30:
Train Loss: 1.1251 | Train Acc: 0.5718
Val Loss: 1.2035 | Val Acc: 0.5634
Epoch 5/30:
Train Loss: 1.0900 | Train Acc: 0.5882
Val Loss: 1.0777 | Val Acc: 0.5968
Saved new best model
Epoch 6/30:
Train Loss: 1.0543 | Train Acc: 0.6031
Val Loss: 1.0451 | Val Acc: 0.5996
Saved new best model
Epoch 7/30:
Train Loss: 1.0260 | Train Acc: 0.6099
Val Loss: 1.0583 | Val Acc: 0.6069
Saved new best model
Epoch 8/30:
Train Loss: 1.0097 | Train Acc: 0.6174
Val Loss: 1.0430 | Val Acc: 0.6141
Saved new best model
Epoch 9/30:
Train Loss: 0.9718 | Train Acc: 0.6327
Val Loss: 1.0258 | Val Acc: 0.6180
Saved new best model
Epoch 10/30:
Train Loss: 0.9505 | Train Acc: 0.6409
Val Lo